In [1]:
using NLOptControl
using Polynomials
using Plots
using Parameters
pyplot()

############
# TEST DATA
############
t0 = Float64(0); tf = Float64(2);
t = Array(linspace(t0,tf,100));

## STATES ##
# sin and cos taylor series approximation at x = 0
s0 =  0; s1 = 1; s2 = 0;       s3 = -1/(3*2*1); s4 = 0;           s5 = 1/(5*4*3*2*1);
c0 = 1;  c1 = 0; c2 = 1/(2*1); c3 = 0;          c4 = 1/(4*3*2*1); c5 = 0;
γ1S = Poly([s0,s1,s2,s3,s4,s5]);  # state 1
γ2S = Poly([c0,c1,c2,c3,c4,c5]);  # state 2
#γS = [γ1S; γ2S]; # ex a 
γS = [γ1S];  # ex b
numStates=length(γS);
yS = [polyval(γS[st],t) for st in 1:numStates];

# evaluate the integrals
∫γS = [polyint(γS[st]) for st in 1:numStates];
YS  = [polyval(∫γS[st],t[end]) - polyval(∫γS[st],t[1]) for st in 1:numStates];
CS  = [YS[st] - polyval(∫γS[st],t[end]) for st in 1:numStates]; # constant of integration
∫yS = [polyval(∫γS[st],t) + CS[st] for st in 1:numStates];

# evaluate the derivatives
dγS = [polyder(γS[st]) for st in 1:numStates];
dyS = [polyval(dγS[st],t) for st in 1:numStates];

## CONTROLS ##
# sin and cos taylor series approximation at x = 0
s0 =  0; s1 = 1; s2 = 0;       s3 = -1/(3*2*1); s4 = 0;           s5 = 1/(5*4*3*2*1);
c0 = 1;  c1 = 0; c2 = 1/(2*1); c3 = 0;          c4 = 1/(4*3*2*1); c5 = 0;
γ1C = Poly([s0,s1,s2,s3,s4,s5]);  # control 1
γ2C = Poly([c0,c1,c2,c3,c4,c5]);  # control 2
#γC = [γ1C; γ2C];  # ex a
γC = [γ2C];   # ex b
numControls=length(γC);
yC = [polyval(γC[ctr],t) for ctr in 1:numControls];

# evaluate the integrals
∫γC = [polyint(γC[ctr]) for ctr in 1:numControls];
YC  = [polyval(∫γC[ctr],t[end]) - polyval(∫γC[ctr],t[1]) for ctr in 1:numControls];
CC  = [YC[ctr] - polyval(∫γC[ctr],t[end]) for ctr in 1:numControls]; # constant of integration
∫yC = [polyval(∫γC[ctr],t) + CC[ctr] for ctr in 1:numControls];

# evaluate the derivatives
dγC = [polyder(γC[ctr]) for ctr in 1:numControls];
dyC = [polyval(dγC[ctr],t) for ctr in 1:numControls];

############
# TEST DATA
############
dx1(t) = 1.0 - 0.5⋅t^2 + 0.0416667⋅t^4;
dx2(t) = 1.0⋅t + 0.166667⋅t^3;
dx = [dx1, dx2];
X0= [yS[1][1];yS[2][1]];
XF= [yS[1][end];yS[2][end]];
XL=[0,-Inf];     XU=[1/9,Inf]; # TODO allow for functions of these so we can calculate them on the fly!
#CL=[-Inf];  CU=[Inf];
CL=[-Inf,-Inf];  CU=[Inf,Inf];

ps, nlp = initialize_NLP(numStates=numStates,numControls=numControls,Ni=5,Nck=[4,4,4,4,4],stateEquations=dx,X0=X0,XF=XF,XL=XL,XU=XU,CL=CL,CU=CU);

@pack ps = t0, tf;  # given in problem def.
@unpack Nck, Ni, t0, tf, τ, ω = ps;
di, tm, ts, ωₛ = create_intervals(t0,tf,Ni,Nck,τ,ω);
@pack ps = τ, ω, ωₛ, ts;
generate_Fake_data(nlp,ps,γS,γC);
nlp2ocp(nlp,ps);

# interpolate function using Lagrange Polynomial | state
@unpack stateMatrix = ps;
@unpack numStates = nlp;
PS = [zeros(Float64,Nck[int]+1,numStates) for int in 1:Ni];
for st in 1:numStates
    for int in 1:Ni
        PS[int][:,st] = interpolate_lagrange(ts[int],ts[int],stateMatrix[int][:,st],Nck[int])
    end
end

# interpolate function using Lagrange Polynomial | control --> had to do Nck[int]-1
@unpack controlMatrix = ps;
@unpack numControls = nlp;
PC = [zeros(Float64,Nck[int]+1,numControls) for int in 1:Ni];
for ctr in 1:numControls
    for int in 1:Ni
        PC[int][:,ctr] = interpolate_lagrange(ts[int],ts[int][1:end-1],controlMatrix[int][:,ctr],Nck[int]-1)
    end
end
# approximate integral using quadrature
stInt1, stIntVal1, ctrInt1, ctrIntVal1 = integrate(ps,nlp)

# calculate LGR matrices - > IMatrix and DMatrix
LGR_matrices(ps,nlp)

# approximate integral using LGRIM
stInt2, stIntVal2, ctrInt2, ctrIntVal2 = integrate(ps,nlp;(:mode=>:LGRIM))

# approximate derivative using LGRDM
dζ = differentiate_state(ps,nlp);

LoadError: BoundsError: attempt to access 1-element Array{Array{Float64,1},1} at index [2]